In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
import re
import random

In [2]:
# To always get the same random results
random.seed(777)

In [3]:
# Load the original dataset and delete null values of price, sellers, stock and category
dataset = pd.read_csv(filepath_or_buffer='originalDataset.csv', encoding="iso-8859-1")
dataset = dataset[dataset['price'].notna()]
dataset = dataset[dataset['sellers'].notna()]
dataset = dataset[dataset['number_available_in_stock'].notna()]
dataset = dataset[dataset['amazon_category_and_sub_category'].notna()]

In [4]:
# Replacing semicoloms in the dataset
dataset['sellers'] = dataset['sellers'].str.replace('\u00A0', ' ');
dataset['amazon_category_and_sub_category'] = dataset['amazon_category_and_sub_category'].str.replace('\u00A0', ' ');
dataset['product_name'] = dataset['product_name'].str.replace('\u00A0', ' ');

In [5]:
# Replacing utf8 spaces for unicode
dataset['sellers'] = dataset['sellers'].str.replace(';',',')
dataset['amazon_category_and_sub_category'] = dataset['amazon_category_and_sub_category'].str.replace(';',',')
dataset['product_name'] = dataset['product_name'].str.replace(';',',')

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4286 entries, 0 to 9998
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      4286 non-null   object 
 1   product_name                                 4286 non-null   object 
 2   manufacturer                                 4284 non-null   object 
 3   price                                        4286 non-null   object 
 4   number_available_in_stock                    4286 non-null   object 
 5   number_of_reviews                            4276 non-null   object 
 6   number_of_answered_questions                 3995 non-null   float64
 7   average_review_rating                        4276 non-null   object 
 8   amazon_category_and_sub_category             4286 non-null   object 
 9   customers_who_bought_this_item_also_bought   3959 non-null   object 
 10  

In [7]:
# dataset.head()

In [8]:
#dataset.tail()

In [9]:
attributes = ['product_name','price','number_available_in_stock','amazon_category_and_sub_category','sellers']
# dataset[attributes]

In [10]:
# ========================================= PREPROCESSING DATASET ==============================================
def removePatterns(str, patterns):
    for pattern in patterns:
        str = re.sub(pattern, '', str)
    return str

def getPrice(str):
    str = str[1:]
    str = str.replace(',','')
    return float(str)

def getUnits(str):
    return int(str.split()[0])

def getCatAndSub(str):
    idxSubCat = str.find(' > ')
    category = str[:idxSubCat]
    subcategory = str[idxSubCat+3:]
    return category, subcategory
    
def getSellers(str):
    # List Initialization
    sellersList = []
    idxList = str.find('[')
    # If there is no array, but one seller only
    if idxList < 0:
        idxList = 10
    # Example: "Seller_name_1"=>"{Amazon.co.uk", "Seller_price_1"=>"£3.42}, {...}, ..., {...}"
    sellersStrList = str[idxList+2:-3].split('}, {')
    for sellerAndPriceStr in sellersStrList:
        # Example: "Seller_name_1"=>"Amazon.co.uk" 
        sellerStr = sellerAndPriceStr.split(', "Seller_price')[0]
        # Example: "Seller_price_1"=>"£3.42"
        priceStr = sellerAndPriceStr.split(', "Seller_price')[1]
        # Example: Amazon.co.uk
        idxAssign = sellerStr.find('=>')
        seller = sellerStr[idxAssign+3:-1]
        # Example: £3.42
        idxAssign = priceStr.find('=>')
        price = getPrice(removePatterns(priceStr, ["[^0-9\.]"]))
        sellersList.append((seller, price))
    return sellersList

def orderBy(dictionary):
    return sorted(dictionary, key=itemgetter('category', 'product', 'units', 'price'))

def groupBy(dictionary, order):
    # Initialize previous tuple
    prev = dictionary[0]
    # For each row in the dictionary
    for row in dictionary:
        # Initialize current row as in the same group as previous
        sameGroup = True
        # Compare each current att to the previous one (for each att in the group criteria)
        for att in order:
            if row[att] != prev[att]: 
                sameGroup = False;
                break;
        # Update the group id
        if sameGroup:
            row["idGroup"] = prev["idGroup"]
        else:
            row["idGroup"] = prev["idGroup"] + 1
        # The current tuple becomes the previous
        prev = row
            

def storeTuple(dictionary, idGroup, product, units, category, sellersAndPrices):
    # EACH INGREDIENT GENERATES A NEW TUPLE
    for sellerAndPrice in sellersAndPrices:        
        seller = sellerAndPrice[0]
        price  = sellerAndPrice[1]
        # random units with maximum value the original units
        randUnits = int(random.random()*units + 1)
        
        tuple = {'idGroup':idGroup, 'seller':seller, 'category':category, 
                 'product':product, 'units':randUnits, 'price':price}
        dictionary.append(tuple)

strRemovals = ["'", '"', "[^A-Za-z0-9\.,\s]"]
dictionary = []
idGroup = 0
for tuple in dataset.iterrows():
    product = removePatterns(tuple[1]['product_name'], strRemovals)
    unitsStr = removePatterns(tuple[1]['number_available_in_stock'], strRemovals)
    units = getUnits(unitsStr)
    catAndSub = removePatterns(tuple[1]['amazon_category_and_sub_category'], strRemovals)
#     category, subcategory = getCatAndSub(catAndSub)
    sellersStr = tuple[1]['sellers']
    sellersAndPrices = getSellers(sellersStr)
    storeTuple(dictionary, idGroup, product, units, catAndSub, sellersAndPrices)
#     idGroup += 1
    
#order and group dictionary
dictionary = orderBy(dictionary)
groupBy(dictionary, ['seller'])

# new data frame
newDataset = pd.DataFrame(dictionary)    

In [11]:
# newDataset.info()
newDataset.head()

,idGroup,seller,category,product,units,price
0,0,AOAO,Arts Crafts Art Sand,10 Minutes Hourglass Sandglass Sand Timer Silv...,2,3.40
1,1,YwYl,Arts Crafts Art Sand,10 Minutes Hourglass Sandglass Sand Timer Silv...,2,5.49
2,2,Hong Kong Wan Xiang,Arts Crafts Art Sand,10 Minutes Hourglass Sandglass Sand Timer Silv...,2,5.49
3,3,Well-Goal,Arts Crafts Art Sand,10 Minutes Hourglass Sandglass Sand Timer Silv...,4,5.49
4,4,Universal Color,Arts Crafts Art Sand,15 Minutes Hourglass Sandglass Sand Timer Blue,1,3.85


In [12]:
# ========================================= STORING DATASET ==============================================
# ======== MY IMPLEMENTATION FORMAT
def getHeader(row, sizes, columns=[]):
    header = ""
    if(len(columns) == 0): header = "id(float,1);"
    currSize = 0
    currType = ""
    i = 0
    for title in row:
        # Verify the column is in the list
        if len(columns) == 0 or title in columns:
            if isinstance(row[title], str):
                currType = "string"
                currSize = sizes[i] + 1
            else:
                currType = "float"
                currSize = 1
            # Add element to header
            header += title + "(" + currType + "," + str(currSize) + ");"
        # Increment
        i += 1
    # printing the header
    return header

def getFormat(value):
    if isinstance(value, str):
        return '"' + str(value) + '"; '
    else:
        return str(value) + "; "
    

def saveData(dictionary, sizes, path = "../T1.data"):
    f = open(path, "w", encoding="iso-8859-1")
    f.write(getHeader(dictionary[0], sizes)+"\n")
    for i in range(len(dictionary)):
        # Id
        f.write(str(i)+"; ")
        for value in dictionary[i].items():
            f.write(getFormat(value[1]))
        f.write("\n")
    f.close()
    
def saveGroups(dictionary):
    f = open("../TG.data", "w")
    for idRow in range(len(dictionary)):
        f.write(str(idRow) + "\t" + str(dictionary[idRow]["idGroup"]) + "\n")
    f.close()

In [13]:
def genUniqueTuples(dictionary, n, otherTable = None, overlap = 0):
    ''' dictionary is a list of tuples having as key the attribute names
        n is the desired size of unique values
        otherTable is for overlapping two tables
        overllap is a number to store keys repeated in both tables
    '''
    size = 0
    uniques = dict()
    while size < n:
        # Get randomly a tuple id
        idTuple = int(random.random()*len(dictionary))
        # Set the key as category and product, those need to be unique
        key = (dictionary[idTuple]['category'], dictionary[idTuple]['product'])
        # Validating that the conditional set has not repeated tuples
        if key not in uniques:
            # If overlap factor is completed and key is not in the other dictionary
            if otherTable == None or (size >= overlap and key not in otherTable):
                uniques[key] = dictionary[idTuple]
                size += 1
            # If overlap factor is not completed and key is in the other dictionary
            elif size < overlap and key in otherTable:
                uniques[key] = dictionary[idTuple]
                size += 1
    # Return the sublist of n unique elements
    return uniques

def saveDataset(storePath, Table, header, columns, limit = None):
    # Open folder in write mode
    f = open(storePath, "w", encoding="iso-8859-1")
    # Writing the header
    f.write(header)
    # Count the number of tuples
    count = 0
    # Save all non limited tuples
    for aTuple in Table.values():
        # Increment the counter of tuples
        count += 1
        # Validate not exceed the limit, if exists
        if limit != None and count > limit:
            break;
        # Iterating columns
        for value in aTuple.items():
            if value[0] in columns:
                if value[0] == 'units':
                    f.write(getFormat(int(value[1])))
                elif value[0] == 'price':
                    f.write(getFormat(float(value[1])))
                else:
                    f.write(getFormat(value[1]))
        # end tuple with end line
        f.write("\n")
    # Close current test file
    f.close()
    
def createTests(dictionary, sizes, columns):
    # General header for T1 or T2
    header = getHeader(dictionary[0], sizes, columns) + "\n"
    # For case 5 (folder for semantics)
    for iTest in range(5,6):
        # The folder path for current test case
        basePath = "../TESTS/TEST " + str(iTest) + "/";
        # Random T2 size between 3 and 6
        sizeT2 = random.randint(3,6)
        # For inside cases (100 test cases)
        for jTest in range(1, 101):
            # add insede folder to path
            path = basePath + str(jTest) + "/"
            # Random T1 size between 3 and 6
            sizeT1 = random.randint(3,6)
            # Generate T1 with unique tuples
            T1 = genUniqueTuples(dictionary, sizeT1)
            # Generate T2 with unique tuples
            T2 = genUniqueTuples(dictionary, sizeT2, T1, int(min(sizeT1, sizeT2)*0.75))
            # Store T1 and T2
            saveDataset(path + "T1.data", T1, header, columns)
            saveDataset(path + "T2.data", T2, header, columns)
            saveDataset(path + "ATUPLE.data", T2, header, columns, 1)
        
        
        

In [14]:
# Max length of strings in the new dataset
measurer = np.vectorize(len)
sizes = measurer(newDataset.values.astype(str)).max(axis=0)
# print(sizes)
# saveData(dictionary, sizes)
# saveGroups(dictionary)
# saveSQLData(dictionary, "T1", sizes)

In [15]:
# The complete cleaned dataset
saveData(dictionary, sizes, "cleanedDataset.csv")

In [16]:
# Create tests 2 - 100
createTests(dictionary, sizes, ['category', 'product', 'units', 'price'])